In [ ]:
import scarlet
import galsim
from astropy import wcs as WCS
import time
from mr_tools import galsim_compare_tools as gct
from mr_tools.simulations import Simulation, chi, SDR
from scarlet.renderer import ResolutionRenderer

# Visualization of resamplings with galsim and scarlet

This notebook can be used to generate random galaxy profiles at various resolutions using galsim. These galaxies are then resampled using scarlet and galsim. The results can be visualized as images for which the Source Distortion Ratio (SDR) compared to the truth, is given.

First let us load the parameters of the surveys we want to emulate.

In [ ]:
def load_surveys():
    """Creates dictionaries for the HST, EUCLID, WFRIST, HCS anf LSST surveys
    that contain their names, pixel sizes and psf fwhm in arcseconds"""
    pix_ROMAN = 0.11
    pix_RUBIN = 0.2
    pix_HST = 0.06
    pix_EUCLID = 0.101
    pix_HSC = 0.167

    #Sigma of the psf profile in arcseconds.
    sigma_ROMAN = 0.11*np.array([1.86]) #https://arxiv.org/pdf/1702.01747.pdf Z-band
    sigma_RUBIN = np.array([0.297]) #https://www.lsst.org/about/camera/features
    sigma_EUCLID = np.array([0.16]) #https://sci.esa.int/documents/33859/36320/1567253682555-Euclid_presentation_Paris_1Dec2009.pdf
    sigma_HST = np.array([0.074]) #Source https://hst-docs.stsci.edu/display/WFC3IHB/6.6+UVIS+Optical+Performance#id-6.6UVISOpticalPerformance-6.6.1 800nm
    sigma_HSC = np.array([0.285]) #https://hsc-release.mtk.nao.ac.jp/doc/ deep+udeep


    EUCLID = {'name': 'Euclid',
              'pixel': pix_EUCLID ,
              'psf': sigma_EUCLID,
              'channels': ['VIS'],
              'sky':np.array([22.9]),
              'exp_time': np.array([2260]),
              'zero_point': np.array([6.85])}
    HST = {'name': 'HST',
           'pixel': pix_HST,
           'psf': sigma_HST,
           'channels': ['f814w'],
           'sky':np.array([22]),
           'exp_time': np.array([3000]),
           'zero_point': np.array([20])}
    HSC = {'name': 'HSC',
           'pixel': pix_HSC,
           'psf': sigma_HSC,
           'channels': ['r'],
           'sky': np.array([20.6]),
           'exp_time': np.array([600]),
           'zero_point': np.array([87.74])}
    ROMAN = {'name': 'Roman',
             'pixel': pix_ROMAN,
             'psf': sigma_ROMAN,
             'channels': ['Y106'],
             'sky':np.array([22]), ## Not Checked!!!
             'exp_time': np.array([3000]),## Not Checked!!!
             'zero_point': np.array([26.41])}
    RUBIN = {'name': 'Rubin',
             'pixel': pix_RUBIN,
             'psf': sigma_RUBIN,
             'channels': ['r'],
             'sky': np.array([21.2]),
             'exp_time': np.array([5520]),
             'zero_point': np.array([43.70])}

    return HST, EUCLID, ROMAN, HSC, RUBIN

The COSMOSCatalog is used to draw images of galaxies. These images being partially noisy, we apply a smoothing by a 2-pixel wide gaussian in the native image resolution (HST).

In [ ]:
%pylab inline
# Setup: declaring survey properties, loading catalog and making sure we have pretty colorbars
data_dir='/Users/remy/Desktop/LSST_Project/GalSim/examples/data'
data_dir='/Users/remy/Desktop/LSST_Project/GalSim/examples/data/COSMOS_23.5_training_sample/'
HST, EUCLID, ROMAN, HSC, RUBIN = load_surveys()

#cat = galsim.COSMOSCatalog(dir=data_dir, file_name = 'real_galaxy_catalog_23.5_example.fits')
cat = galsim.COSMOSCatalog(dir=data_dir, file_name = 'real_galaxy_catalog_23.5.fits')

mymap = 'gnuplot2'#mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)
matplotlib.rc('image', cmap='gist_stern')
matplotlib.rc('image', interpolation='none')

Setup for the image shapes and scarlet meta parameters

In [ ]:
# Choose to surveys to match
survey_hr = HSC
survey_lr = RUBIN

# PSF size (pixels)
npsf = 41

# The low resolution image will span the same physical area
n_lr = 60
n_hr = np.int(np.around(n_lr*survey_lr['pixel']/survey_hr['pixel'], decimals = 3))
print(n_lr, n_hr)
# Channel names (scarlet-specific)
channel_hr = ['hr']
channel_lr = ['lr']
channels = channel_lr+channel_hr

Plotting routines:

In [ ]:
def plot_diff_kernels(true_psf, scarlet_rec, galsim_rec):
    # Comparisonbetween scarlet and galsim of the reconstruction of the low resolution PSF at high resolution 
    r_scar=(true_psf.array-scarlet_rec.image)[0]
    figure(figsize = (15,5))
    plt.suptitle('comparison scarlet', fontsize = 20)
    subplot(131)
    title('scarlet PSF')
    imshow((scarlet_rec.image[0]))
    plt.colorbar()
    subplot(132)
    title('true PSF')
    imshow((true_psf.array))
    plt.colorbar()
    subplot(133)
    title('Residuals')
    imshow((r_scar), cmap = mymap, 
           vmin = -np.max(np.abs(r_scar)), vmax = np.max(np.abs(r_scar)))
    plt.colorbar()

    r_gal = (true_psf.array-galsim_rec.array)
    figure(figsize = (15,5))
    plt.suptitle('comparison galsim', fontsize = 20)
    subplot(131)
    title('galsim PSF')
    imshow((galsim_rec.array))
    plt.colorbar()
    subplot(132)
    title('true PSF')
    imshow((true_psf.array))
    plt.colorbar()
    subplot(133)
    title('Residuals')
    imshow((r_gal), cmap = mymap, 
           vmin = -np.max(np.max(r_gal)), vmax = np.max(np.max(r_gal)))
    plt.colorbar()
    pass

def plot_comparison(true_hr, true_lr, scarlet_lr, galsim_lr):
    # Source Distortion Ratio: the high the better
    scarlet_sdr = SDR(data_lr, interp_scar[0])
    galsim_sdr = SDR(data_lr, interp_gal)
    print('scarlet SDR: ', scarlet_sdr)
    print('galsim SDR: ', galsim_sdr)
    
    # Comparison of the reconstructions 
    gal_res = true_lr-galsim_lr
    scar_res = true_lr-scarlet_lr[0]
    figure(figsize = (10,15))
    plt.suptitle('Interpolation comparison', fontsize=30)
    subplot(321)
    title('True '+survey_hr['name'], fontsize = 20)
    imshow(((true_hr)))
    plt.colorbar()
    subplot(322)
    title('True '+survey_lr['name'], fontsize = 20)
    imshow(((true_lr)))
    plt.colorbar()
    subplot(323)
    title('Scarlet interpolated', fontsize = 20)
    imshow(((scarlet_lr[0])))
    plt.text(35,5,f'SDR = {str(scarlet_sdr)[:6]}', color = 'w', fontsize = 12)
    plt.colorbar()
    subplot(324)
    title('Galsim interpolation', fontsize = 20)
    imshow(((galsim_lr)))
    plt.text(35,5,f'SDR = {str(galsim_sdr)[:6]}', color = 'w', fontsize = 12)
    plt.colorbar()
    subplot(325)
    title('Scarlet residuals', fontsize = 20)
    imshow((scar_res), cmap=mymap, vmin = -np.max(np.abs(scar_res)), vmax = np.max(np.abs(scar_res)))
    plt.colorbar()
    subplot(326)
    title('Galsim residuals', fontsize = 20)
    imshow((gal_res), cmap=mymap, )
    #vmin = -np.max(np.abs(scar_res)), vmax = np.max(np.abs(scar_res)))
    plt.colorbar()
    
    plt.savefig('Resampling_example_'+survey_hr['name']+'_'+survey_lr['name']+'.png')
    show()
    pass

## Generate sims

Simulations of low and high resolution of the same galaxy are generated using the mr_tools package, which uses galsim to generate galaxy images from cosmos real galaxies.

In [ ]:
# Make the simulations
pic_hr, pic_lr = gct.mk_scene(survey_hr,
                          survey_lr,
                          cat,
                          (n_hr,n_hr),
                          (n_lr,n_lr),
                          1, 
                          gal_type = 'real', 
                          random_seds = False, 
                          noise = False,
                          use_cat = False,
                          shift = False,
                          index = 9775)
wcs_hr = pic_hr.wcs
wcs_lr = pic_lr.wcs

data_hr = pic_hr.cube[0]/np.sum(pic_hr.cube[0])
data_lr = pic_lr.cube[0]/np.sum(pic_lr.cube[0])


psf_hr = scarlet.ImagePSF(np.array(pic_hr.psfs)[None, :,:])
psf_lr = scarlet.ImagePSF(np.array(pic_lr.psfs)[None, :,:])

ref = wcs_hr.wcs.crval.reshape((1,2))
    

## Scarlet setup

We build the scarlet frames that constructs the operator to resample an image from a resolution to another.

In [ ]:
# Setup scarlet
obs, frame = gct.setup_scarlet(data_hr, data_lr, wcs_hr, wcs_lr, psf_hr, psf_lr, channels, 'intersection')
obs_lr, obs_hr = obs
    

## Building the galsim diff kernel

To resample an image from one resolution to another, one has to interpolate between pixel positions, but also needs to take into account the difference between the psfs. 
- In the scarlet scheme these two operation (interpolation and psf-matching) are handled internally as one operation. 
- In the galsim case, we need to build the difference kernel between the high and low resolution psfs, drawn on the same grid.

We show how accurately each method (galsim or scarlet) reconstructs the low resolution psf from the high resolution image and the diff kernel.

In [ ]:
# Galsim setup:
## Interpolated image from psf_hr for galsim
psf_hr_galsim = galsim.InterpolatedImage(galsim.Image(psf_hr.get_model()[0]), 
                                           scale = survey_hr['pixel'])

## Interpolated Image of the low resolution galaxy for galsim
psf_lr_galsim = galsim.InterpolatedImage(galsim.Image(psf_lr.get_model()[0]), 
                                           scale = survey_lr['pixel'])

#diff kernel
## Deconvolution kernel for diff kernel
deconv = galsim.Deconvolve(psf_hr_galsim)
## Building the diff kernel in galsim
diff_gal = galsim.Convolve(deconv, psf_lr_galsim)

# Check on the reconstruction of the PSF
## Reconstruction of the low resolution psf by galsim from hr psf and diff kernel (sanity check)
rec_gal = galsim.Convolve(diff_gal, psf_hr_galsim).drawImage(nx=npsf,
                                                             ny=npsf, 
                                                             method = 'real_space', 
                                                             scale=survey_hr['pixel'])

#Extract the difference kernel for scarlet (for sanity checks)
## Pull the renderer
renderer = ResolutionRenderer(obs_lr, frame)
## Difference kernel from scarlet 
diff_scar = obs_lr.renderer.diff_kernel

## True psf
true_lr = pic_lr.psfs_obj[0].drawImage(nx=npsf,
                                    ny=npsf,
                                    method = 'real_space',
                                    scale=survey_hr['pixel'])

## Reconstruction of the low resolution psf by SCARLET from hr psf and diff kernel (sanity check)
rec_scar = scarlet.fft.convolve(scarlet.fft.Fourier(psf_hr.get_model()), 
                                scarlet.fft.Fourier(diff_scar.image))

plot_diff_kernels(true_lr, rec_scar, rec_gal)

## Results

We compare the reconstructions from galsim and scarlet. We also record the time each strategy spends on the reconstruction. The operations we time here are the only operations that need to be repeated in an iterative problem-solving approach. The steps of interpolating the PSF for galsim and building the resampling operator for scarlet are performed beforehand.

In [ ]:
# Run scarlet on simulations:
print('Scarlet timing')
%time renderer(data_hr[None,:,:])
interp_scar = renderer(data_hr[None, :,:])

# Run galsim on simlations
print('Galsim timing')
%time  gct.interp_galsim(data_hr, data_lr, diff_gal, 0, survey_hr['pixel'], survey_lr['pixel'])
interp_gal = gct.interp_galsim(data_hr, data_lr, diff_gal, 0, survey_hr['pixel'], survey_lr['pixel']).array

In [ ]:
plot_comparison(data_hr, data_lr, interp_scar, interp_gal)